# DEA Coastlines generation using command line tools <img align="right" src="https://github.com/GeoscienceAustralia/dea-notebooks/raw/develop/Supplementary_data/dea_logo.jpg">
This notebook demonstrates how to run a DEA Coastlines analysis using command line tools.

### Setup

Set working directory to top level of repo to ensure links work correctly:

In [1]:
cd ..

/home/jovyan/Robbi/dea-coastlines


Update required packages:

In [2]:
pip install -r requirements.in --quiet

You should consider upgrading via the '/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Set analysis parameters:

In [6]:
study_area = 64
raster_version = 'testing'
vector_version = 'testing'
continental_version = 'testing'
config_path = 'configs/dea_coastlines_config.yaml'

# study_area = 9
# raster_version = 'development'
# vector_version = 'development'
# continental_version = 'development'
# config_path = 'configs/dea_coastlines_config_development.yaml'

### Run DEA Coastlines analysis
#### Run tidally-constrained raster generation

In [3]:
!python -m coastlines.raster --help

Usage: python -m coastlines.raster [OPTIONS]

Options:
  --config_path TEXT              Path to the YAML config file defining inputs
                                  to use for this analysis. These are
                                  typically located in the `dea-
                                  coastlines/configs/` directory.  [required]
  --study_area TEXT               A string providing a unique ID of an
                                  analysis gridcell that will be used to run
                                  the analysis. This should match a row in the
                                  "id" column of the provided analysis
                                  gridcell vector file.  [required]
  --raster_version TEXT           A unique string proving a name that will be
                                  used for output raster directories and
                                  files. This can be used to version different
                                  analysis outputs.  [req

Example analysis:

In [6]:
!python -m coastlines.raster --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1988 --end_year 2021

<Client: 'tcp://127.0.0.1:40609' processes=1 threads=62, memory=477.21 GiB>
2023-01-30 06:17:55 INFO Study area 151: Loaded study area grid
2023-01-30 06:18:04 INFO Study area 151: Loaded virtual product
Creating reduced resolution tide modelling array
Modelling tides using FES2014 tide model
Reprojecting tides into original array
100%|███████████████████████████████████████| 1021/1021 [00:30<00:00, 33.01it/s]
2023-01-30 06:19:44 INFO Study area 151: Finished modelling tide heights
2023-01-30 06:19:45 INFO Study area 151: Calculating low and high tide cutoffs for each pixel
2023-01-30 06:19:45 INFO Study area 151: Started exporting raster data
/env/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/env/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
2023-01-30 06:31:57

#### Run vector annual shoreline and rates of change statistics generation

In [4]:
!python -m coastlines.vector --help

Usage: python -m coastlines.vector [OPTIONS]

Options:
  --config_path TEXT              Path to the YAML config file defining inputs
                                  to use for this analysis. These are
                                  typically located in the `dea-
                                  coastlines/configs/` directory.  [required]
  --study_area TEXT               A string providing a unique ID of an
                                  analysis gridcell that was previously used
                                  to generate raster files. This is used to
                                  identify the raster files that will be used
                                  as inputs for shoreline extraction, and
                                  should match a row in the "id" column of the
                                  provided analysis gridcell vector file.
                                  [required]
  --raster_version TEXT           A unique string providing a name that was
   

Example analysis:

In [7]:
!python -m coastlines.vector --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1988 --end_year 2021 --baseline_year 2021

2023-01-31 01:56:17 INFO Study area 64: Starting vector generation
2023-01-31 01:56:29 INFO Study area 64: Loaded rasters
Operating in single z-value, multiple arrays mode
2023-01-31 01:57:02 INFO Study area 64: Extracted shorelines from raster data
2023-01-31 01:57:02 INFO Study area 64: Extracted rates of change points
2023-01-31 01:57:26 INFO Study area 64: Calculated distances to each annual shoreline
2023-01-31 01:57:28 INFO Study area 64: Calculated rates of change regressions
2023-01-31 01:57:31 INFO Study area 64: Calculated all of time statistics
2023-01-31 01:57:31 INFO Study area 64: Calculated rate of change certainty flags
2023-01-31 01:57:32 INFO Study area 64: Added region attributes and geohash UIDs
2023-01-31 01:57:39 INFO Study area 64: Output vector files written to data/interim/vector/testing/64_testing


#### Run continental-scale layer generation

In [ ]:
!python -m coastlines.continental --help

Example analysis:

In [ ]:
!python -m coastlines.continental --vector_version {vector_version} --continental_version {continental_version} --shorelines True --ratesofchange True --hotspots True --baseline_year 2021

## Example combined analysis
This demonstrates how the three components of DEA Coastlines (raster generation, vector generation and continental layers generation) can be applied automatically to a sequence of input study area grid cells.

In [ ]:
# Study areas
study_areas = [193, 478]

In [ ]:
# Run raster and vector generation for each study area
for study_area in study_areas:
    print(study_area)
    !python -m coastlines.raster --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1988 --end_year 2021
    !python -m coastlines.vector --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --vector_version {vector_version} --start_year 1988 --end_year 2021 --baseline_year 2021

# When complete, combine into single continental outputs
# !python -m coastlines.continental --vector_version {vector_version} --continental_version {continental_version} --shorelines True --ratesofchange True --hotspots True --baseline_year 2021

### Clean up and remove files
Warning: this will delete all data processed with a matching unique version name. Only run these cells if you no longer need these processed outputs!

In [ ]:
# rm -rf data/interim/raster/{raster_version}

In [ ]:
# rm -rf data/interim/vector/{vector_version}

In [ ]:
# rm -rf data/processed/{continental_version}

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** For assistance with any of the Python code or Jupyter Notebooks in this repository, please post a [Github issue](https://github.com/GeoscienceAustralia/dea-coastlines/issues/new).

**Last modified:** November 2022

**To cite:**

> Bishop-Taylor, R., Nanson, R., Sagar, S., Lymburner, L. (2021). Mapping Australia's dynamic coastline at mean sea level using three decades of Landsat imagery. Remote Sensing of Environment, 267, 112734. Available: https://doi.org/10.1016/j.rse.2021.112734
>
> Nanson, R., Bishop-Taylor, R., Sagar, S., Lymburner, L., (2022). Geomorphic insights into Australia's coastal change using a national dataset derived from the multi-decadal Landsat archive. Estuarine, Coastal and Shelf Science, 265, p.107712. Available: https://doi.org/10.1016/j.ecss.2021.107712
>
> Bishop-Taylor, R., Sagar, S., Lymburner, L., Alam, I., Sixsmith, J. (2019). Sub-pixel waterline extraction: characterising accuracy and sensitivity to indices and spectra. Remote Sensing, 11 (24):2984. Available: https://doi.org/10.3390/rs11242984